In [ ]:
# Low resolution cases for tests
import os
import glob
import itertools
import numpy as np
# switch for spinup and drainage scripts

if_spinup = True
if_drainage = True
case = 1

# delete previous generated scripts
# Delete all files in the specified directories
for directory in ['./generated_scripts/spinup', './generated_scripts/drainage', './generated_scripts/output']:
    files = glob.glob(f"{directory}/*")
    for file in files:
        os.remove(file)

if case == 1:
    # Reference case
    moulin_inputs = [0, 10]
    mus = [1e1]
    kappas = [1e-10]
    Vls_log10 = [7]
elif case == 2:
    # Dependence of velocity on viscosity
    moulin_inputs = [0]  
    mus = [1e-3,1e-2,1e-1,1e0,1e1,1e2,1e3,1e4]
    kappas = [1e-10]
    Vls_log10 = [7]
elif case == 3:
    # Dependence of velocity on volume
    moulin_inputs = [0]  
    mus = [1e1]
    kappas = [1e-10]
    Vls_log10 = [4,5,6,7,8]
elif case == 4:
    # Dependence of volume on kappa
    moulin_inputs = [10]  
    mus = [1e3]
    kappas = [1e-12, 1e-11, 1e-10, 1e-9, 1e-8, 1e-7, 1e-6]
    Vls_log10 = [7]

# parameter combinations (moulin input, viscosity, kappa, log10(volume))
param_combos = [
    # (0, 1e-3, 1e-20, 6), # propagation case with varying viscosities and volumes
    # (0, 1e-3, 1e-20, 7),
    # (0, 1e-3, 1e-20, 8),
    # (0, 1e-3, 1e-20, 9),
    # (0, 1e-2, 1e-20, 6),
    # (0, 1e-2, 1e-20, 7),
    # (0, 1e-2, 1e-20, 8),
    # (0, 1e-2, 1e-20, 9),
    # (0, 1e-1, 1e-20, 6),
    # (0, 1e-1, 1e-20, 7),
    # (0, 1e-1, 1e-20, 8),
    # (0, 1e-1, 1e-20, 9),
    # (0, 1e0, 1e-20, 6),
    # (0, 1e0, 1e-20, 7),
    # (0, 1e0, 1e-20, 8),
    # (0, 1e0, 1e-20, 9),
    # (0, 1e1, 1e-20, 6), 
    # (0, 1e1, 1e-20, 7),
    # (0, 1e1, 1e-20, 8),
    # (0, 1e1, 1e-20, 9),
    # (0, 1e2, 1e-20, 6),
    # (0, 1e2, 1e-20, 7),
    # (0, 1e2, 1e-20, 8),
    # (0, 1e2, 1e-20, 9),
    # (0, 1e3, 1e-20, 6),
    (0, 1e3, 1e-20, 7),
    # (0, 1e3, 1e-20, 8),
    # (0, 1e3, 1e-20, 9),
    # (0, 1e1, 1e-10, 7), # reference cases
    # (10, 1e1, 1e-10, 7),
    # (100, 1e1, 1e-10, 7),
    # (10, 1e3, 1e-10, 7), # wintertime leakage case with varying kappas and mu->infty
    # (10, 1e3, 1e-11, 7),
    # (10, 1e3, 1e-12, 7),
    # (10, 1e3, 1e-13, 7),
    # (10, 1e3, 1e-14, 7),
    # (10, 1e3, 1e-15, 7),
    # (10, 1e3, 1e-16, 7),
    # (10, 1e3, 2e-10, 7),
    # (10, 1e3, 3e-10, 7),
    # (10, 1e3, 4e-10, 7),
    # (10, 1e3, 5e-10, 7),
    # (10, 1e3, 6e-10, 7),
    # (10, 1e3, 7e-10, 7),
    # (10, 1e3, 8e-10, 7),
    # (10, 1e3, 9e-10, 7),
    # (10, 1e1, 1e-8, 7), # summertime leakage case with varying kappas and mu->infty
    # (10, 1e1, 1e-9, 7),
    # (10, 1e1, 1e-10, 7),
    # (10, 1e1, 1e-11, 7),
    # (10, 1e1, 1e-12, 7),
    # (10, 1e1, 1e-13, 7),
]

# param_combos = [
#     (0, 1e1, 1e-10, 7),
#     (10, 1e1, 1e-10, 7),
# ]
Nx = 401
Ny = 160


out_dir = './generated_scripts/spinup'
os.makedirs(out_dir, exist_ok=True)

spinup_template = '''%% Script to run NEVIS regional model 
% This script is designed to run the NEVIS 1-dimensional model for an idealised ice sheet
% The surface runoff is set up by a prescribed function
% The surface and bed profiles are the same as in Hewitt 2013

% Author: Hanwen Zhang  
% Date: 2025-05
format compact

%% setup paths and directories
clear,clc
oo.root = './';                                % filename root
oo.code = '../nevis/src';                      % code directory  
oo.results = 'results';                        % path to the results folders
oo.dataset = 'nevis_regional';                 % dataset name  
oo.casename = '{casename}';                    % casename
oo.fn = ['/',oo.casename];                     % filename (same as casename)
oo.rn = [oo.root,oo.results,oo.fn];            % path to the case results
oo.dn = [oo.root, 'data/', oo.dataset, '/'];   % path to the data
addpath(oo.code);                              % add path to code
mkdir(oo.rn);                                  % create directory for results    

%% parameters
% default parameters 
[pd,oo] = nevis_defaults([],oo);  

oo.evaluate_variables = 1;
oo.input_gaussian = 1;
oo.relaxation_term = 1;                         % 0 is alpha hb, 1 is alpha deltap hb
oo.initial_condition = 1;                       % 1 is default condition from 0365.mat, 0 is using steady-state drainage system, winter or summertime
oo.mean_perm = 1;
oo.display_residual =0;
% leakage term
if oo.relaxation_term == 0                      % 0: exponential decay: -\\alpha_0(1+h/hc+S/Sc) h_b         
    pd.alpha_b = 1.0/(10*pd.td);                % relaxation rate (s^-1)
    pd.kappa_b = 0;                             % relaxation coeff 
    pd.c0 = 1;
elseif oo.relaxation_term == 1                  % 1: proportional to pressure diff and thickness: -\\kappa/\\mu(p_b-p_w)h_b
    pd.alpha_b = 0;                             % relaxation rate (s^-1)
    pd.kappa_b = {kappa};                      % relaxation coeff
    pd.c0 = 1;
end

% alter default parmaeters 
runoff_max = 30;                                % prescribed runoff (mm/day)
moulin_input = {moulin_input};                  % prescribed moulin input (m^3/s)
pd.mu = {mu};                                  % water viscosity (Pa s)
pd.Ye = 8.8e9;                                  % Young's modulus (Pa)
pd.B = pd.Ye*(1e3)^3/(12*(1-0.33^2));           % bending stiffness (Pa m^3)
pd.E_lapse = 30/1000/pd.td/10^3;

pd.hb_reg1 = 5e-3;                              % Regularisation parameter for hb
pd.hb_reg2 = 1e-3;                              % Regularisation parameter for hb
pd.N_reg1 = 1e4;                                % Regularisation parameter for N
pd.deltap_reg = 1e4;                            % Regularisation parameter for deltap
pd.B_reg = 0;                                   % Regularisation parameter for B

pd.G = 0.01;                                    % geothermal heat flux [J/s/m^2]
pd.melt = pd.G/pd.rho_w/pd.L;                   % geothermal heat derived basal melt [m/s]
                              
% non-dimensionalise
ps = struct;
[ps,pp] = nevis_nondimension(pd,ps,oo);   

%% grid and geometry
L = 5e4;                                     % length of the domain [m]
W = 0.4*L;                                   % width of the domain [m]
x = linspace(0,(L/ps.x),{Nx}); 
y = linspace(0,(W/ps.x),{Ny});        
oo.yperiodic = 1;                        % oo.yperiodic = 1 necessary for a 1-d grid
oo.xperiodic = 0;
gg = nevis_grid(x,y,oo); 
b = (0/ps.z)*gg.nx;                      % flat bed
s = (1060/ps.z)*(1-ps.x*gg.nx/L).^0.5;   % ice surface topography 

%% mask with minimum ice thickness
H = max(s-b,0);
Hmin = 0/ps.z; 
nout = find(H<=Hmin);
gg = nevis_mask(gg,nout); 
gg.n1m = gg.n1;                                   % label all edge nodes as boundary nodes for pressure

%% label boundary nodes
gg = nevis_label(gg,gg.n1m);
oo.adjust_boundaries = 1;                         % enable option of changing conditions

%% plot grid
% nevis_plot_grid(gg); return;                    % check to see what grid looks like

%% initialize variables
[aa,vv] = nevis_initialize(b,s,gg,pp,oo);         % default initialisation
pd.k_f = 1.00;                                    % percent overburden (k-factor) 
vv.phi = aa.phi_a+pd.k_f*(aa.phi_0-aa.phi_a);     % initial pressure  k_f*phi_0
N = aa.phi_0-vv.phi;                              % N for initial cavity sheet size 
vv.hs = ((((pd.u_b*pd.h_r/pd.l_r)./((pd.u_b/pd.l_r)+(pd.K_c.*((ps.phi*N).^3)))))./ps.h); % initial cavity sheet size as f(N)

%% boundary conditions
aa.phi_b = max(aa.phi_0,aa.phi_a);                % prescribed boundary pressure at overburden or atmospheric

%% moulins 
oo.keep_all_moulins = 0;
oo.random_moulins = 0;         
[pp.ni_m,pp.sum_m] = nevis_moulins([0.25*L/ps.x],[0.5*W/ps.x],gg,oo);     % one moulin at the lake location

%% supraglacial lakes
pp.x_l = [0.5*L/ps.x];                                          % x-coord of lakes
pp.y_l = [0.5*W/ps.x];                                          % y-coord of lakes
pp.V_l = [0e7/(ps.Q0*ps.t)];                                    % volume of lakes         
pp.t_drainage = [0.5*365*pd.td/ps.t];                           % time of lake drainages
pp.t_duration = [0.025*pd.td/ps.t];                              % duration of lake drainages
[pp.ni_l,pp.sum_l] = nevis_lakes(pp.x_l,pp.y_l,gg,oo);          % calculate lake catchments 

%% surface input
oo.surface_runoff = 0;                          
oo.RACMO_runoff = 0;                            
oo.distributed_input = 0;                       
pp.meltE = @(t) (runoff_max/1000/pd.td/ps.m)*(1-exp(-t/(30*pd.td/ps.t)));
pp.input_function = @(t) moulin_input*(1-exp(-t/(30*pd.td/ps.t)))./(ps.m*ps.x^2);     

%% Timestep 
oo.dt = 1/24*pd.td/ps.t; 
oo.save_timesteps = 1; 
oo.save_pts_all = 1; 
oo.pts_ni = [pp.ni_l pp.ni_m];                      
oo.t_span = (1:1:2*365)*pd.td/ps.t;                 

%% save initial parameters
save([oo.rn, oo.fn],'pp','pd','ps','gg','aa','vv','oo');
[tt,vv,info] = nevis_timesteps(oo.t_span,vv,aa,pp,gg,oo);

%% expand/update variables
aa = nevis_inputs(vv.t,aa,vv,pp,gg,oo);
oo.evaluate_variables = 1; 
[vv2] = nevis_backbone(inf,vv,vv,aa,pp,gg,oo); 
vv2 = nevis_nodedischarge(vv2,aa,pp,gg,oo); 
save([oo.rn, oo.fn],'pp','pd','ps','gg','aa','oo','tt');

%% Simple animate
% nevis_regional_animation
'''

# Generate spinup scripts
for m, mu, kappa, Vl_log10 in param_combos:
    mu_parts = f"{mu:.0e}".split('e')
    mu_mantissa = mu_parts[0].replace('.', '')
    mu_exp = mu_parts[1].replace('-', '_').replace('+', '')
    
    # 格式化 kappa: "2e-10" -> "2e_10" 
    kappa_parts = f"{kappa:.0e}".split('e')
    kappa_mantissa = kappa_parts[0].replace('.', '')
    kappa_exp = kappa_parts[1].replace('-', '_').replace('+', '')

    casename =  f"n2d_{m}m3s_kappa{kappa_mantissa}e{kappa_exp}_mu{mu_mantissa}e{mu_exp}_hbreg5e_3_spinup"
    filename = os.path.join(out_dir, f"{casename}.m")
    if if_spinup:
        with open(filename, 'w') as f:
            f.write(spinup_template.format(
                casename=casename,
                moulin_input=m,
                mu=mu,
                kappa=kappa,
                Nx=Nx,
                Ny=Ny,
            ))
print(f"Generation completed: {len(os.listdir(out_dir))} spinup scripts in `{out_dir}` directory.")
# print spinup scripts in the directory
print("Generated spinup scripts:")
for filename in os.listdir(out_dir):
    print(f" - {filename}")


# spinup_dir = './generated_scripts/spinup'
out_dir = './generated_scripts/drainage'
os.makedirs(out_dir, exist_ok=True)

# Clean up old files
import glob
old_files = glob.glob(os.path.join(out_dir, "*.m"))
for f in old_files:
    os.remove(f)

drainage_full_template = '''%% Script to run NEVIS regional model 
% This script is designed to run the NEVIS 1-dimensional model for an idealised ice sheet
% The surface runoff is set up by a prescribed function
% The surface and bed profiles are the same as in Hewitt 2013

% Author: Hanwen Zhang  
% Date: 2025-05
format compact

%% read in the initial condition
casename = '{drainage_case}'; % drainage system filename
initname = strrep(casename, '_V1e{Vl_log10}_drainage', '_spinup'); % initial condition filename

data = load(['./results/' initname '/' initname]);
pd = data.pd;                                % load parameters from the initial condition
ps = data.ps;                                % load state variables from the initial condition
pp = data.pp;                                % load scaled parameters from the initial condition
aa = data.aa;                                % load state variables from the initial condition
oo = data.oo;                                % load options from the initial condition

oo.casename = casename;                      % drainage system filename
oo.initname = initname;                      % initial condition filename, for spinup

oo.fn = ['/',oo.casename];                     % filename (same as casename)
oo.rn = [oo.root,oo.results,oo.fn];            % path to the case results
oo.dn = [oo.root, 'data/', oo.dataset, '/'];   % path to the data
addpath(oo.code);                              % add path to code
mkdir(oo.rn);                                  % create directory for results    
pp.c0 = 0;

%% grid and geometry
L = 5e4;                                     % length of the domain [m]
W = 0.4*L;                                   % width of the domain [m]
x = linspace(0,(L/ps.x),{Nx}); 
y = linspace(0,(W/ps.x),{Ny});        
oo.yperiodic = 1;                            % oo.yperiodic = 1 necessary for a 1-d grid
oo.xperiodic = 0;
gg = nevis_grid(x,y,oo); 
b = (0/ps.z)*gg.nx;                          % flat bed
s = (1060/ps.z)*(1-ps.x*gg.nx/L).^0.5;       % ice surface topography 

%% mask with minimum ice thickness
H = max(s-b,0);
Hmin = 0/ps.z; 
nout = find(H<=Hmin);
gg = nevis_mask(gg,nout); 
gg.n1m = gg.n1;                                   % label all edge nodes as boundary nodes for pressure

%% label boundary nodes
gg = nevis_label(gg,gg.n1m);
oo.adjust_boundaries = 1;                         % enable option of changing conditions

%% plot grid
% nevis_plot_grid(gg); return;                    % check to see what grid looks like

%% initialize variables
init_cond = load(['./results/' oo.initname '/' '0730.mat']); % load initial condition
vv = init_cond.vv;                                % load state variables from the initial condition

%% supraglacial lakes
pp.x_l = [0.5*L/ps.x];                                          
pp.y_l = [0.5*W/ps.x];                                          
pp.V_l = [1e{Vl_log10}/(ps.Q0*ps.t)];                           % different V_l values
pp.t_drainage = vv.t+ [20*pd.td/ps.t];                          % time of lake drainages
pp.t_duration = [0.025*pd.td/ps.t];                             % duration of lake drainages
[pp.ni_l,pp.sum_l] = nevis_lakes(pp.x_l,pp.y_l,gg,oo);         % calculate lake catchments 

%% timestep 
oo.dt = 1/24*pd.td/ps.t; 
oo.save_timesteps = 1; 
oo.save_pts_all = 1; 
% Add GPS station points downstream of the moulin every 5km
pp.ni_gps = nevis_gps_array((pp.x_l:5e3/ps.x:L/ps.x), pp.y_l*ones(size((pp.x_l:5e3/ps.x:L/ps.x))), gg, oo); % GPS station points
oo.pts_ni = [pp.ni_l pp.ni_m pp.ni_gps];                                                
oo.t_span = vv.t + [(1:1:19)*pd.td/ps.t (19.9:0.001:20.1)*pd.td/ps.t (21:1:1.5*365)*pd.td/ps.t];            

%% save initial parameters
save([oo.rn, oo.fn],'pp','pd','ps','gg','aa','vv','oo');
[tt,vv,info] = nevis_timesteps(oo.t_span,vv,aa,pp,gg,oo);

%% expand/update variables
aa = nevis_inputs(vv.t,aa,vv,pp,gg,oo);
oo.evaluate_variables = 1; 
[vv2] = nevis_backbone(inf,vv,vv,aa,pp,gg,oo); 
vv2 = nevis_nodedischarge(vv2,aa,pp,gg,oo); 
save([oo.rn, oo.fn],'pp','pd','ps','gg','aa','oo','tt');

%% Simple animate
% nevis_regional_animation
'''

# Generate drainage scripts
for m, mu, kappa, Vl_log10 in param_combos:

    mu_parts = f"{mu:.0e}".split('e')
    mu_mantissa = mu_parts[0].replace('.', '')
    mu_exp = mu_parts[1].replace('-', '_').replace('+', '')
    
    kappa_parts = f"{kappa:.0e}".split('e')
    kappa_mantissa = kappa_parts[0].replace('.', '')
    kappa_exp = kappa_parts[1].replace('-', '_').replace('+', '')

    spinup = f"n2d_{m}m3s_kappa{kappa_mantissa}e{kappa_exp}_mu{mu_mantissa}e{mu_exp}_hbreg5e_3"

    # Ensure only integers are used here
    suffix = f"V1e{Vl_log10}"  # Vl_log10 is already an integer 5,6,7,8
    casename = f"{spinup}_{suffix}_drainage"
    fname = os.path.join(out_dir, f"{casename}.m")
    if if_drainage:
        with open(fname, 'w') as f:
            f.write(drainage_full_template.format(
                drainage_case=casename, 
                Vl_log10=Vl_log10,
                Nx=Nx,
                Ny=Ny
            ))

print("Generation completed!", len(os.listdir(out_dir)), " .m scripts generated in", out_dir)

# Only show correctly named files
print("Generated drainage scripts:")
for i, filename in enumerate(sorted(os.listdir(out_dir))):
    if '+' not in filename:
        print(f" - {filename}")

Generation completed: 1 spinup scripts in `./generated_scripts/spinup` directory.
Generated spinup scripts:
 - n2d_0m3s_kappa1e_20_mu1e03_hbreg5e_3_spinup.m
Generation completed! 1  .m scripts generated in ./generated_scripts/drainage
Generated drainage scripts:
 - n2d_0m3s_kappa1e_20_mu1e03_hbreg5e_3_V1e7_drainage.m
